In [13]:
import os

In [14]:
%pwd

'/home/shrey/Desktop/Kidney-Disease-Classifcation'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/shrey/Desktop/Kidney-Disease-Classifcation'

In [15]:
os.chdir("src")

In [16]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_include_top: bool
    params_weights: str
    params_classes: int

In [17]:
from kidney_classification.constants import *
from kidney_classification.utils.common import read_yaml, create_directories

In [18]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES
        )

        return prepare_base_model_config

In [19]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, BatchNormalization, Dropout
from tensorflow.keras.models import Sequential

In [26]:
class PrepareBaseModel:
    @staticmethod
    def prepare_full_model():
        VGG_model = Sequential()

        pretrained_model= tf.keras.applications.VGG16(
            include_top=False,
            input_shape=(150, 150, 3),
            pooling='max',
            classes=4,
            weights='imagenet'
        )

        VGG_model.add(pretrained_model)
        VGG_model.add(Flatten())
        VGG_model.add(Dense(512, activation='relu'))
        VGG_model.add(BatchNormalization())
        VGG_model.add(Dropout(0.5))

        VGG_model.add(Dense(4, activation='softmax'))
        pretrained_model.trainable = False

        VGG_model.compile(
            optimizer='adam',
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy']
        )

        return VGG_model

    def update_base_model(self, config: PrepareBaseModelConfig):
        full_model = self.prepare_full_model()

        full_model.summary()
        full_model.save(config.updated_base_model_path)
    
        
    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)

In [22]:
os.chdir("../")

In [27]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel()
    prepare_base_model.update_base_model(prepare_base_model_config)
except Exception as e:
    raise e

[2024-01-03 15:58:32,485: INFO: common yaml file: config/config.yaml loaded successfully]
[2024-01-03 15:58:32,487: INFO: common yaml file: params.yaml loaded successfully]
Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 512)               14714688  
                                                                 
 flatten_3 (Flatten)         (None, 512)               0         
                                                                 
 dense_5 (Dense)             (None, 512)               262656    
                                                                 
 batch_normalization_3 (Bat  (None, 512)               2048      
 chNormalization)                                                
                                                                 
 dropout_3 (Dropout)         (None, 512)               0         
             